In [6]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from PIL import Image

train_transform = transforms.Compose([
    transforms.Resize(384, interpolation=Image.BICUBIC),
    transforms.CenterCrop(384),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize(384, interpolation=Image.BICUBIC),
    transforms.CenterCrop(384),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder('../data/splited/train', transform=train_transform)
val_dataset = datasets.ImageFolder('../data/splited/valid', transform=test_transform)

batch_size = 16
num_workers = 2
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [ ]:
model = models.regnet_y_32gf(weights="IMAGENET1K_SWAG_E2E_V1")
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

class_weights = torch.tensor([1.0 / len(train_dataset.classes) for _ in range(len(train_dataset.classes))]).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

optimizer = optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.5)

train_loss_values = []
val_loss_values = []
accuracy_values = []
n_epochs = 5
for epoch in range(n_epochs):
    current_loss = {'train': 0, 'val': 0}
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        current_loss['train'] += loss.item() * inputs.size(0)
        loss.backward()
        optimizer.step()
    current_loss['train'] /= len(train_loader.dataset)
    train_loss_values.append(current_loss['train'])

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            current_loss['val'] += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        current_loss['val'] /= len(val_loader.dataset)
        val_loss_values.append(current_loss['val'])
        scheduler.step()
    accuracy = 100 * correct / total
    accuracy_values.append(accuracy)

    print('Epoch [{:3d}] Train loss: {:.4f} / Valid loss: {:.4f} / Valid accuracy: {:.2f}%'.format
            (epoch + 1, current_loss['train'], current_loss['val'], accuracy))

torch.save(model.state_dict(), 'best.pt')

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_loss_values, label='Training Loss')
plt.plot(val_loss_values, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(accuracy_values, label='Validation Accuracy', color='green')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.show()